In [1]:
import random
from uuid import uuid1
import timeit

In [2]:
class Car:
    def __init__(self):
        self.type = 'Car'
        self.id = str(uuid1())
        self.locations = [Location(self) for x in range(0, 10)]
        
class Person:
    def __init__(self):
        self.type = 'Person'
        self.id = str(uuid1())
        self.cars = []
        
    def add_car(self, car):
        self.cars.append(car)
        
class Location:
    def __init__(self, car):
        self.type = 'Location'
        self.id = str(uuid1())
        self.lat = random.randint(0, 100)
        self.lng = random.randint(0, 100)
        self.car = car

In [3]:
persons = [Person() for x in range(0, 10)]
cars = [Car() for x in range(0, 10)]

for person in persons:
    [person.cars.append(random.choice(cars)) for x in range(0,3)]

In [4]:
from py2neo import Graph, Path, Node, Relationship, NodeMatcher
graph = Graph()
matcher = NodeMatcher(graph)

# clear db
graph.evaluate("MATCH (n) DETACH DELETE n")

In [9]:
# Insert cars
for car in cars:
    tx = graph.begin()
    
    car_node = Node('Car', id=car.id)
    location_nodes = [Node('Location', id=location.id, lat=location.lat, lng=location.lng) for location in car.locations]
    tx.create(car_node)
    
    for location_node in location_nodes:
        tx.create(location_node)
        rel = Relationship(car_node, 'WAS_AT', location_node)
        tx.create(rel)
        
    tx.commit()

print(len(persons))
# Insert people
for person in persons:
    tx = graph.begin()
    person_node = Node('Person', id=person.id)
    tx.create(person_node)
    
    for car in person.cars:
        car_node = matcher.match('Car', id=car.id).first()
        rel = Relationship(person_node, 'OWNED', car_node)
        tx.create(rel)
    tx.commit()

10


In [29]:
def neo4j_query():
    car= random.choice(cars)
    graph.evaluate("match (:Car {id: '" + car.id + "'}) <-[:OWNED]- (p:Person) -[:OWNED]-> (c:Car) -[:WAS_AT]-> (l:Location) return p, c, l")

timeit.timeit(stmt=neo4j_query, number=20)

0.2904170439869631

In [32]:
from pymongo import MongoClient

mongo = MongoClient()
mongodb = mongo['test-db']
person_collection = mongodb['people']
car_collection = mongodb['cars']

car_collection.drop()
person_collection.drop()

In [33]:
for person in persons:
    person_node = {
        'type': 'Person',
        'id': person.id,
        'cars': [car.id for car in person.cars]
    }
    person_collection.insert_one(person_node)
    
for car in cars:
    car_node = {
        'type': 'Car',
        'id': car.id,
        'locations': [{
            'lat': location.lat,
            'lng': location.lng
        } for location in car.locations]
    }
    car_collection.insert_one(car_node)

In [43]:
def mongo_query():
    car = random.choice(cars)
    person = person_collection.find({"cars": car.id})[0]
    
    locations = []
    
    for car_id in person['cars']:
        [locations.append(x) for x in car_collection.find({"id": car_id})]
    
timeit.timeit(stmt=mongo_query, number=20)

ace7beb6-6d5d-11e8-95f9-9cb6d0d3a297
ace78a18-6d5d-11e8-95f9-9cb6d0d3a297
ace73e28-6d5d-11e8-95f9-9cb6d0d3a297
ace72a0a-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace74ed6-6d5d-11e8-95f9-9cb6d0d3a297
ace76920-6d5d-11e8-95f9-9cb6d0d3a297
ace76920-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace71240-6d5d-11e8-95f9-9cb6d0d3a297
ace779f6-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace76920-6d5d-11e8-95f9-9cb6d0d3a297
ace779f6-6d5d-11e8-95f9-9cb6d0d3a297
ace7beb6-6d5d-11e8-95f9-9cb6d0d3a297
ace7ae94-6d5d-11e8-95f9-9cb6d0d3a297
ace7beb6-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace76920-6d5d-11e8-95f9-9cb6d0d3a297
ace779f6-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace76920-6d5d-11e8-95f9-9cb6d0d3a297
ace779f6-6d5d-11e8-95f9-9cb6d0d3a297
ace79c42-6d5d-11e8-95f9-9cb6d0d3a297
ace76920-6d5d-11e8-95f9-9cb6d0d3a297
ace779f6-6d5d-11e8-95f9-9cb6d0d3a297
a

0.1493966720008757